In [444]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

## Classes

In [445]:
class Individual:
  def __init__(self,c, w, fitness=math.inf):
    self.c = c
    self.w = w
    self.fitness = fitness

  # Getters
  def getFitness(self):
    return self.fitness

  # Setters
  def setFitness(self,f):
    self.fitness = f
  
  # Methods
  def printIndividual(self):
    return f'C (size): {self.c.shape}, W (size): {self.w.shape}, fitness: {self.fitness}\nC: {self.c}\nW: {self.w}'.format(self=self)
  
  def createIndividual(K,D):
    return Individual(random_C_generator(K), random_W_generator(K,D))

In [446]:
class Population:
  def __init__(self,size):
    self.size = size
    self.geneList = []
    self.fitnessList = []
    self.bestGene = None

  # Methods
  def printPopulation(self):
    print(f'Population size: {len(self.geneList)}, best fitness: {self.bestGene.fitness}')
  def addGeneToList(self,gen):
    self.geneList.append(gen);

  def addFitnessToList(self,fitness):
    self.fitnessList.append(fitness);

  def insertBestGene(self,gen):
    self.bestGene = gen
    self.geneList.append(gen)
    self.fitnessList.append(gen.fitness)

  def updateBestGene(self, gen):
    self.bestGene = gen

  def createPopulation(self,K,D):
    for i in range(self.size):
      gen = Individual(random_C_generator(K), random_W_generator(K,D))
      self.geneList.append(gen)
    
    # Once the population is set, we choose a random best gene
    randomIndex = np.random.randint(low=0, high=self.size)
    self.bestGene = self.geneList[randomIndex]
    return self

In [447]:
class Particle(Individual):
  def __init__(self,c, w, fitness=math.inf):
    Individual.__init__(self, c, w, fitness)
    self.bestFitness = fitness
    self.wPosition = w
    self.cPosition = c
    self.wVelocity = 0 # All swarm's velocities are set to 0 at the start
    self.cVelocity = 0 

  # Setters
  def updateWeightsPosition(self, new_W):
    self.wPosition = new_W
    
  def updateChromosomePosition(self, new_C):
    self.cPosition = new_C

  def updateWeightsVelocity(self, new_W):
    self.wVelocity = new_W

  def updateChromosomeVelocity(self, new_C):
    self.cVelocity = new_C

  def updateBestFitness(self,new_fitness):
    self.bestFitness = new_fitness

In [448]:
class Swarm(Population):
  def __init__(self,size):
    Population.__init__(self, size)
    self.globalBestW = None
    self.globalBestC = None

  # Getters
  def getBestSwarmWeights(self):
    return self.globalBestW

  def getBestSwarmChromosome(self):
    return self.globalBestC

  # Setters
  def setBestSwarmWeigths(self,newW):
    self.globalBestW = newW

  def setBestSwarmChromosome(self,newC):
    self.globalBestC = newC

  def initializeSwarm(self):
    for i in range(self.size):
      p = Particle(random_C_generator(K), random_W_generator(K,optimalD))
      self.geneList.append(p)
          
    # Once the swarm is set, we choose a random best gene
    randomIndex = np.random.randint(low=0, high=self.size)
    self.bestGene = self.geneList[randomIndex]
    self.globalBestC = self.bestGene.c
    self.globalBestW = self.bestGene.w  
    return self

In [449]:
class Reef:
  def __init__(self,rho0, R):
    self.reefSize = R
    self.freeOccupiedRate = rho0
    self.geneList = np.full([R,1], None)
    self.fitnessList = []
    self.bestGene = None

  # Methods
  def addGeneToList(self,gen):
    self.geneList.append(gen);

  def addFitnessToList(self,fitness):
    self.fitnessList.append(fitness);

  def insertBestGene(self,gen):
    self.bestGene = gen
    self.geneList.append(gen)
    self.fitnessList.append(gen.fitness)

  def initializeReef(self):
    occupiedHoles = int(self.reefSize * self.freeOccupiedRate)# Partially Occupied
    print(occupiedHoles)
    for i in range(occupiedHoles): 
      # Choose random hole
      random_hole_index = np.random.randint(0,self.geneList.shape[0])
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if not self.geneList[random_hole_index][0]:
          break
        else:
          random_hole_index = np.random.randint(0,self.geneList.shape[0])

      # Insert Individual in that hole
      larvae = Individual(random_C_generator(K), random_W_generator(K,optimalD))
      self.geneList[random_hole_index][0] = larvae
      
      # Once the population is set, we choose a random best gene
      random_best_hole_index = np.random.randint(0,self.geneList.shape[0])
      
      while 1: # If that hole is occupied then choose again until it finds an empty one 
        if not self.geneList[random_best_hole_index][0]:
          self.bestGene = self.geneList[random_best_hole_index][0]
          break
        else:
          random_best_hole_index = np.random.randint(0,self.geneList.shape[0])

    return self

## Functions

### NumPy custom functions

In [450]:
def sumMatrixes(mat1,mat2):
  return np.add(mat1,np.asarray(mat2))

def subtractMatrixes(mat1,mat2):
  return np.subtract(mat1,np.asarray(mat2))

def dotMultiply(mat1,mat2):
  return np.multiply(mat1,np.asarray(mat2))

def multiplyMatrixes(mat1,mat2):
  return np.matmul(mat1,np.asarray(mat2))

def dotDivide(mat1,mat2):
  return np.divide(mat1,np.asarray(mat2))

### Population

In [451]:
def random_C_generator(K):
  # randint returns from: inclusive -[low,high)- exclusive.
  return np.random.randint(low= 0, high= 1 + 1, size=[K])

def random_W_generator(K,D):
  return np.random.uniform(low= -1, high= 1, size=[K,D])

#### Reproduction

In [452]:
def InternalReproduction(gen):
  # Mutate chromosome 
  mutate_point = np.random.randint(0, len(gen.c)) # Choose a random point to mutate
  gen.c[mutate_point] = 1 - gen.c[mutate_point] # Mutate the point
  
  # Mutate weights
  nRows = gen.w.shape[0] 
  nColumns = gen.w.shape[1] 
  mutate_column = np.random.randint(0, nColumns) # Choose a random column to mutate
  
  gen.w[:,mutate_column] = np.random.uniform(-1,1,nRows) # Mutate the column

In [453]:
def ExternalReproduction(father,mother):
  crossover_c_point = np.random.randint(0, len(father.c))
  crossover_column_w_point = random.randint(0, len(father.w[0]))
  
  # Offsprings
  offspring1 = Individual( c = np.concatenate((father.c[:crossover_c_point], mother.c[crossover_c_point:])), 
                           w = np.concatenate((father.w[:,:crossover_column_w_point], mother.w[:,crossover_column_w_point:]), axis=1) ) 
  
  offspring2 = Individual( c = np.concatenate((mother.c[:crossover_c_point], father.c[crossover_c_point:])), 
                           w = np.concatenate((mother.w[:,:crossover_column_w_point], father.w[:,crossover_column_w_point:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [454]:
'''     REPRODUCTION TESTS PASSED     '''

'''
populationTest = Population(size=10).createPopulation(K=4,D=3)

hijosTest = ExternalReproduction(populationTest.geneList[0],populationTest.geneList[1])

for index, i in enumerate(hijosTest):
    print(f"Son:{index}{i.c}")

print("Before:",populationTest.geneList[0].c)
print("Before:",populationTest.geneList[0].w)
InternalReproduction(populationTest.geneList[0])

print("After:",populationTest.geneList[0].c)
print("After:",populationTest.geneList[0].w)
'''

'\npopulationTest = Population(size=10).createPopulation(K=4,D=3)\n\nhijosTest = ExternalReproduction(populationTest.geneList[0],populationTest.geneList[1])\n\nfor index, i in enumerate(hijosTest):\n    print(f"Son:{index}{i.c}")\n\nprint("Before:",populationTest.geneList[0].c)\nprint("Before:",populationTest.geneList[0].w)\nInternalReproduction(populationTest.geneList[0])\n\nprint("After:",populationTest.geneList[0].c)\nprint("After:",populationTest.geneList[0].w)\n'

In [455]:
def RouletteWheelSelection(population): 
  # Create a population for the chosen individuals of the Roulette
  roulettePopulation = Population(population.size)

  # Make sure we don't lose our best gene in the roulette 
  roulettePopulation.insertBestGene(population.bestGene)

  # Total population fitness (S)
  S = sum([chromosome.fitness for chromosome in population.geneList])
  
  # Population chromosomes' relative probabilities
  rel_prob = [chromosome.fitness/S for chromosome in population.geneList]

  for i in range(population.size - 1):
    # Generate a random uniform number (r) - (0,1]
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.geneList): 
      r -= rel_prob[index]
      if r < 0: 
        roulettePopulation.addGeneToList(individual) 
        roulettePopulation.addFitnessToList(individual.fitness) 
        break

  return roulettePopulation  

In [456]:
# This functions reproduces a population.
# It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
# Returns the input population, updated with the crossover and mutated childs

def ReproducePopulation(population):
  childs = [] # list to store future offsprings
  crossoverProb = np.random.uniform() # crossover probability
  mutationProb = np.random.uniform() # mutation probability
  geneListSize = len(population.geneList)

  for index,individual in enumerate(population.geneList):
    # Generate a random number and check if it's over the crossover probability
    if(random.random() >= crossoverProb):
      random_mother_index = np.random.randint(0,geneListSize)

      while(random_mother_index == index): # Make sure that the mother is a different individual in the population
        random_mother_index = np.random.randint(0,geneListSize)

      mother = population.geneList[random_mother_index]

      # Crossover parents
      offsprings = ExternalReproduction(individual, mother)

      # Mutate offsprings
      if(random.random() < mutationProb): 
        InternalReproduction(offsprings[0])

      if(random.random() < mutationProb):
        InternalReproduction(offsprings[1])

      childs.append(offsprings[0])
      childs.append(offsprings[1])
  
  # Update population's Genes list with newly-created childs
  population.geneList = np.concatenate((population.geneList, childs))


  return population

In [457]:
'''    FINAL REPRODUCTION FUNCTION TESTS PASSED     '''
'''
populationTest = Population(size=10).createPopulation(K=9,D=2)

# Print population before reproduction
populationTest.printPopulation()

# Reproduce population
populationReproduced = ReproducePopulation(populationTest)

# Print population before reproduction
populationReproduced.printPopulation()
'''

'\npopulationTest = Population(size=10).createPopulation(K=9,D=2)\n\n# Print population before reproduction\npopulationTest.printPopulation()\n\n# Reproduce population\npopulationReproduced = ReproducePopulation(populationTest)\n\n# Print population before reproduction\npopulationReproduced.printPopulation()\n'

#### Swarm

In [458]:
def UpdateVelocitiesAndPositionsPSO(swarm: Swarm, w:float, c1:float, c2:float):
  for index, p in enumerate(swarm.geneList):
    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    
    # Update particle's velocities applying PSO's equations
    p.wVelocity = (w * p.wVelocity) + (c1*r1*(p.bestFitness - p.wPosition)) + (c2*r2*(swarm.bestGene.wPosition - p.wPosition))
    
    # Print all parameters in previous equation
    print("- - - - - - - - ")
    p.cVelocity = (w * p.cVelocity) + (c1*r1*(p.bestFitness - p.cPosition)) + (c2*r2*(swarm.bestGene.cPosition - p.cPosition))
    print("p.cPosition: \n",p.cPosition)
    print("Swarm.bestGene.cPosition: \n",swarm.bestGene.cPosition)
    print("p.cVelocity:\n ",p.cVelocity)

    print(f"Particle {index} W velocity:", p.wVelocity)
    print(f"Particle {index} C velocity:",p.cVelocity)
    print("- - - - - - - - ")

    return swarm
    # Positions
    velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*cVt_i) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1
    if np.random.uniform() < velocityProbability:
      cPt_i = 1
    else:
      cPt_i = 0

    wPt_i = p.wPosition + wVt_i

    print("wPt_i:", wPt_i)
    # -- for END --

## Data Processing

In [459]:
CSV_NAME = "car-evaluation.csv"
CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/" + CSV_NAME
df = pd.read_csv(CSV_PATH, sep=" ", header=None)

end = df.shape[1]
X = df.iloc[:, :end-1] # iloc[rows,[cols_start,cols_end)] <- Dataframe object
Y = df.iloc[:, end-1] # iloc[rows,col_index] <- Dataframe series

# IMPORTANT, otherwise we would be using DataFrames and matrix operations won't work
X = X.to_numpy()
Y = Y.to_numpy()
J = len(np.unique(Y))

N,K = X.shape[0],X.shape[1] 

In [460]:
# Scaling X (min-max normalization)
X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# First partition training/test
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # 20% test data

# Second partition validation
X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # 20% validation data

## ELM

In [461]:
# Outputs the fitness for a given Gene
def ComputeChromosomeFitness(individual, D,C,trainingX, trainingY, testX, testY):
    # W
    individualWeights = individual.w

    # Feature-selection Bias given Gene's chromosome
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Output
    Y_predicted = multiplyMatrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [462]:
#testChromosome = Individual.createGenome(10,3)

In [463]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputePopulationFitness(population, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  population.fitnessList.clear()

  for index,individual in enumerate(population.geneList):
    f_i = ComputeChromosomeFitness(individual, D, C, trainingX, trainingY, testX, testY)
     
      # Set fitness
    individual.setFitness(f_i)
    #print("Best Gene Fi middle:", population.bestGene.fitness)
    population.addFitnessToList(f_i)

      # Update Best Gene in Population
    if(individual.fitness < population.bestGene.fitness):
      population.bestGene = individual

In [464]:
# Updates the fitnesses of a given population of Genes and also, set the new best Gene 
def ComputeSwarmFitness(swarm: Swarm, D,C,trainingX, trainingY, testX, testY):
  # Clear previous fitnesses
  swarm.fitnessList.clear()

  for index,particle in enumerate(swarm.geneList):
    f_i = ComputeChromosomeFitness(particle, D, C, trainingX, trainingY, testX, testY)

    # Set particle's fitness
    particle.setFitness(f_i)
    swarm.addFitnessToList(f_i)

    # Update Pb_i for each particle and find the best gene 
    if particle.fitness < particle.bestFitness:
      particle.bestFitness = particle.fitness
      
      if particle.fitness < swarm.bestGene.fitness:
        swarm.bestGene = particle

  # Update Gbest of the swarm
  swarm.setBestSwarmChromosome(swarm.bestGene.c)
  swarm.setBestSwarmWeigths(swarm.bestGene.w)
  print("Global best fitness: ", swarm.bestGene.fitness)

In [465]:
def TrainModelAndOutputResults(weigths, D,C,trainingX, trainingY, testX, testY):
      # W
    W = weigths 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, W) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, W) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiplyMatrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

## Genetic Algorithm

In [466]:
def GeneticAlgorithm(gaPopulation, nGenerations, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < nGenerations:
      # Reproduction
    ReproducePopulation(gaPopulation)  
      # Compute Fitness
    ComputePopulationFitness(gaPopulation, optimalD, optimalC, trainX, trainY, testX, testY)

    print("Generation: ", t, " Best Fitness: ", gaPopulation.bestGene.fitness)
      # Selector operator
    gaPopulation = RouletteWheelSelection(gaPopulation)
      # Continue iterating
    t += 1
  
  return gaPopulation

In [467]:
  # GA
  sizePopulation = 7
  nGenerations = 5
  crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population functino
  mutation_prob = 0.15
  optimalD = 1000
  optimalC = 1

  gaStartExTime = time.time()
  print("- - - - - - - -")
  print("Starting GA ...")
  
  # Init population  
  gaFirstGeneration = Population(sizePopulation).createPopulation(K,optimalD)

  # Apply Algorithm and Evolve
  lastGeneration = GeneticAlgorithm(gaFirstGeneration, nGenerations, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
  print("Best fitness:", lastGeneration.bestGene.fitness)

  # Print lastGeneration best gene
  print("Best feature selection: ", lastGeneration.bestGene.c)
  print("Best Weights: ", lastGeneration.bestGene.w)
  # Train ELM with best W
  GA_CCR = TrainModelAndOutputResults(lastGeneration.bestGene.w, optimalD, optimalC, X_train, Y_train, X_test, Y_test)
  
  print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
  print("GA execution time: ", time.time() - gaStartExTime, "sec")
  print("- - - - - - - -")

- - - - - - - -
Starting GA ...
Generation:  0  Best Fitness:  8.651292577625123
Generation:  1  Best Fitness:  8.546124828034879
Generation:  2  Best Fitness:  8.545223258957884
Generation:  3  Best Fitness:  8.624041639895456
Generation:  4  Best Fitness:  8.598275803421886
Best fitness: 8.598275803421886
Best feature selection:  [0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0]
Best Weights:  [[-0.45950853 -0.23215473 -0.17923906 ...  0.86257856  0.58618949
  -0.13641034]
 [-0.97864792  0.69389643 -0.84009519 ... -0.54393565 -0.42667105
  -0.85979503]
 [ 0.93795318  0.19348854  0.78551128 ...  0.94638463 -0.41818767
   0.07208316]
 ...
 [-0.25053687 -0.41677148 -0.5341769  ...  0.72983968 -0.00223051
  -0.52170647]
 [ 0.30581879 -0.59869623  0.11645743 ... -0.88436724 -0.93232298
   0.51362511]
 [-0.30423998 -0.00440626  0.73824142 ...  0.85226526  0.58951274
  -0.73806309]]
GA CCR: 70.8092485549133 
Error: 29.190751445086704
GA execution time:  9.303722381591797 sec
- - - - - - - -


## Particle Swarm Optimization

In [468]:
def PSO_Algorithm(swarm, maxGenerations, w, c1, c2, dampingW, optimalD, optimalC, trainX, trainY, testX, testY):
  t = 0
  while t < maxGenerations:
    # Evaluate Particles' Fitness and update Pb_i and Gb_i
    ComputeSwarmFitness(swarm, optimalD, optimalC, trainX, trainY, testX, testY)
    
    # Update Velocity and Position of each particle
    UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2)

    # Damping inertia
    w = w * dampingW

    # Increase step
    t += 1

  return 1

In [469]:
# PSO
swarm_size = 3 # total population
num_generations = 10
w = 1 # inertia weight coefficient
dampingW = 0.99 # damping for inertia coefficient
c1 = 2 # cognitive coefficient
c2 = 2 # social coefficient
optimalD = 1000
optimalC = 1

print("Starting PSO method...")
pso_start_execution_time = time.time()
# Init population
swarm = Swarm(swarm_size).initializeSwarm() # Swarm velocity is set to 0 by default
# Apply Algorithm and Iterate
PSO_Algorithm(swarm, num_generations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

print("PSO Completed")

print("Best solution found:")
print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")


Starting PSO method...
Global best fitness:  8.859369838157491
- - - - - - - - 
p.cPosition: 
 [0 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1]
Swarm.bestGene.cPosition: 
 [0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1]
p.cVelocity:
  [15.79804807 15.79804807 14.03964138 16.88072492 16.88072492 12.95696453
 16.88072492 15.79804807 16.88072492 12.95696453 16.88072492 12.95696453
 12.95696453 16.88072492 14.03964138 12.95696453 15.79804807 14.03964138
 16.88072492 12.95696453 14.03964138]
Particle 0 W velocity: [[16.93015622 13.56222828 18.12761992 ... 16.59828276 18.62416968
  15.94639141]
 [17.62153181 14.67912316 17.97148279 ... 16.23972332 18.31750601
  14.13777629]
 [17.42000274 14.25419708 17.12104636 ... 18.61127926 14.00118668
  15.5098236 ]
 ...
 [18.75622169 17.52069086 13.45210187 ... 15.61495278 15.34902752
  15.6049709 ]
 [16.96198487 14.58252815 14.32911987 ... 17.89104087 15.72851573
  15.86075788]
 [16.52954628 14.15730021 16.73711948 ... 12.88578459 17.96149817
  19.44906368]]
Pa

## Coral Reef Optimization

In [470]:
def CROAlgorithm():
  t = 0
  return 1
  '''
  while t < coralGenerations:
    # Update variables
    F_broadcast = np.random.uniform() # broadcast fraction
    F_brooding = 1 - F_broadcast # brooding fraction
    Pd = 0.2 # predation probability
    
    # Broadcast and Brooding
    
    # Asexual reproduction

    # Predation 

    # Fitness Evaluation

    # Increase step
    t += 1
    # - end WHILE

  print("Best CRO: ", "coral X")
  '''

In [471]:
# CRO
reefSize = 10
rho0 = 0.6 # free/occupied rate
eta = 2 # larva's attempts
coralGenerations = 10 
F_broadcast = np.random.uniform() # broadcast fraction
F_brooding = 1 - F_broadcast # brooding fraction
Pd = 0.2 # predation probability

croStartExTime = time.time()
print("- - - - - - - -")
print("Starting CRO method...")
# Init population
reefFirstGeneration = Reef(rho0,reefSize).initializeReef()

print(reefFirstGeneration.geneList)
# Apply Algorithm and Evolve
#PSOAlgorithm(swarm, iterations, w, c1, c2, dampingW, optimalD, optimalC, X_trainVal, y_trainVal, X_testVal, y_testVal)

print("CRO Completed")

print("CRO execution time: ", time.time() - croStartExTime, "sec")
print("- - - - - - - -")

- - - - - - - -
Starting CRO method...
6
[[<__main__.Individual object at 0x0000015FFF18CA60>]
 [None]
 [None]
 [None]
 [None]
CRO Completed
CRO execution time:  0.003000497817993164 sec
- - - - - - - -


# MAIN

In [472]:
if __name__ == '__main__':
  # ! We should set random number generator seed
  #np.random.seed(seed=0)

  # - - ALGORITHMS - - -
  print("Hello guys...")


Hello guys...
